In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llamabot.code_manipulation import get_git_diff
from pyprojroot import here

diff = get_git_diff()
print(diff)

In [ ]:
from llamabot import SimpleBot

commitbot = SimpleBot("You are an expert user of Git.")

from outlines import text


@text.prompt
def commit_message(diff: str):
    """Please write a commit message for the following diff.

    {{ diff }}
    """

In [ ]:
commitbot(commit_message(diff))

In [ ]:
import fnmatch
from pathlib import Path


def should_ignore_file(file_path: Path, gitignore_patterns: list[str]) -> bool:
    for pattern in gitignore_patterns:
        if fnmatch.fnmatch(file_path.name, pattern):
            return True
    return False


def print_directory_tree(
    directory: str,
    depth: int = float("inf"),
    indent: int = 0,
    ignore_gitignore: bool = False,
) -> str:
    directory_path = Path(directory)
    if not directory_path.is_dir():
        raise NotADirectoryError(
            f"The specified path '{directory}' is not a directory."
        )

    if depth < 0:
        return ""

    files = []
    folders = []

    # Collect files and folders
    for entry in directory_path.iterdir():
        if entry.is_file():
            files.append(entry)
        elif entry.is_dir():
            folders.append(entry)

    # Sort files and folders alphabetically
    files.sort()
    folders.sort()

    # Read gitignore patterns
    gitignore_patterns = []
    gitignore_path = directory_path / ".gitignore"
    if ignore_gitignore and gitignore_path.is_file():
        with gitignore_path.open("r") as gitignore_file:
            gitignore_patterns = gitignore_file.read().splitlines()

    # Prepare the printed text
    printed_text = ""

    # Print files
    for file in files:
        if ignore_gitignore and should_ignore_file(file, gitignore_patterns):
            continue
        printed_text += "|" + "    " * indent + "|-- " + file.name + "\n"

    # Recursively print subdirectories
    for folder in folders:
        printed_text += "|" + "    " * indent + "|-- " + folder.name + "/" + "\n"
        if depth > 0:
            printed_text += print_directory_tree(
                folder, depth - 1, indent + 1, ignore_gitignore
            )

    return printed_text

In [ ]:
from pyprojroot import here

file_tree = print_directory_tree(here(), depth=0, ignore_gitignore=True)
print(file_tree)

In [ ]:
from outlines import text


@text.prompt
def directory_structure_tutorial(file_tree):
    """Please explain the following directory structure.

    {{ file_tree }}

    You should cover the purpose for why each file exists.

    Use bullet points, not numbered lists.
    """

In [ ]:
from llamabot import SimpleBot

bot = SimpleBot("You are an expert in Python data science.")

file_tree_explanations = bot(directory_structure_tutorial(file_tree))

In [ ]:
@text.prompt
def explain_jargon(text):
    """For the following text, please identify any jargon
    that may be unfamiliar to a junior data scientist
    and write out definitions for each of them.

    {{ text }}

    Begin your response with:

    ## Jargon Explanation
    """

In [ ]:
jargon_explanations = bot(explain_jargon(file_tree_explanations.content))